In [ ]:
import torch
import torch.nn as nn
import torchvision.datasets as ds
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor, Normalize, Compose
import matplotlib.pyplot as plt
from tqdm.auto import trange, tqdm
print(torch.__version__)

MNIST = ds.MNIST(root="./data", train=True, download=True, transform=Compose([ToTensor(), Normalize([0.5], [0.5])]))
dataloader_train = DataLoader(MNIST, batch_size=64, shuffle=True)
MNIST2 = ds.MNIST(root="./data", train=False, download=True, transform=Compose([ToTensor(), Normalize([0.5], [0.5])]))
dataloader_test = DataLoader(MNIST2, batch_size=64, shuffle=True)
print(len(MNIST))

device = "cuda" if torch.cuda.is_available() else "cpu"
batch, labels = next(iter(dataloader_train))
print(batch.shape)
print(labels.shape)

plt.figure(figsize=[15,15])
for i in range(64):
    plt.subplot(8,8,i+1)
    plt.imshow(batch[i].squeeze().detach().numpy())
    plt.title(labels[i].item())
plt.show()


In [2]:
class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 8, 3)
        self.conv2 = nn.Conv2d(8, 16, 3)
        self.conv3 = nn.Conv2d(16, 8, 3)
        self.bn1 = nn.BatchNorm2d(8)
        self.bn2 = nn.BatchNorm2d(16)
        self.bn3 = nn.BatchNorm2d(8)
        self.act = nn.ReLU()
        self.pool = nn.MaxPool2d(2)
        self.fc = nn.Linear(4*4*8, 10)

    def forward(self, x):
        out1 = self.bn1(self.pool(self.act(self.conv1(x))))
        out2 = self.bn2(self.act(self.conv2(out1)))
        out3 = self.bn3(self.pool(self.act(self.conv3(out2))))
        out = self.fc(out3.flatten(start_dim=-3, end_dim=-1))
        return out1, out2, out3, out
        


In [ ]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
model = CNN().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
n_epochs = 3
criterion = nn.CrossEntropyLoss()
for epoch in range(n_epochs):
    #print(f"Epoch {epoch+1}/{n_epochs}")
    model.train()
    l = 0
    a = 0
    trainbar = tqdm(dataloader_train)
    count = 0
    for img, label in trainbar:
        optimizer.zero_grad()
        img, label = img.to(device), label.to(device)
        *_, output = model(img)
        #print(output.shape)
        loss = criterion(output, label)
        l += loss.item()
        loss.backward()
        optimizer.step()
        acc = torch.sum(torch.argmax(output, dim=-1)==label).item()
        a += acc
        count += len(img)
        trainbar.set_description(f"Training Epoche {epoch+1}/{n_epochs}, Loss: {l/count:.4e}, Acc: {a/count*100:.2f}%")
    
    model.eval()
    l = 0
    a = 0
    testbar = tqdm(dataloader_test)
    count = 0
    for img, label in testbar:
        img, label = img.to(device), label.to(device)
        with torch.no_grad():
            *_, output = model(img)
        loss = criterion(output, label)
        l += loss.item()
        acc = torch.sum(torch.argmax(output, dim=-1)==label).item()
        a += acc
        count += len(img)
        testbar.set_description(f"Testing Epoche {epoch+1}/{n_epochs}, Loss: {l/count:.4e}, Acc: {a/count*100:.2f}%")


In [ ]:
# --- Hier kann der Index geändert werden
# Beispiel:
# idx = 1: 0
# idx = 3: 1
# idx = 5: 2
# idx = 7: 3
# idx = 2: 4
# idx = 0: 5
# idx = 13: 6
# idx = 15: 7
# idx = 17: 8
# idx = 4: 9

idx = 3
# -----
img, label = MNIST[idx]

out1, out2, out3, output = model(img.to(device).unsqueeze(0))
plt.figure()
plt.imshow(img[0].detach().cpu().numpy())
plt.suptitle("Eingabebild")
plt.show()

plt.figure()
for i in range(8):
    plt.subplot(2,4,i+1)
    plt.imshow(out1.squeeze()[i].detach().cpu().numpy())
plt.suptitle("Output Layer 1")
plt.show()

plt.figure()
for i in range(16):
    plt.subplot(4,4,i+1)
    plt.imshow(out2.squeeze()[i].detach().cpu().numpy())
plt.suptitle("Output Layer 2")
plt.show()

plt.figure()
for i in range(8):
    plt.subplot(2,4,i+1)
    plt.imshow(out3.squeeze()[i].detach().cpu().numpy())
plt.suptitle("Output Layer 3")
plt.show()

plt.figure(figsize=[12,5])
plt.subplot(1,2,1)
plt.plot(output.squeeze().detach().cpu().numpy())
plt.title("Output")
plt.subplot(1,2,2)
plt.plot(torch.nn.functional.softmax(output.squeeze(), dim=0).detach().cpu().numpy())
plt.title("Output (Softmax)")
plt.show()

In [ ]:
print(dir(model.conv1))
kernellist = model.conv1.weight
plt.figure()
for i in range(8):
    plt.subplot(2,4,i+1)
    plt.imshow(kernellist[i,0].detach().cpu().numpy())
plt.show()